In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc='apply')
import dill
import os
import sys

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# %matplotlib inline

In [2]:
def _iter_lines(files):
    for file in tqdm(files, desc='feathers'):
        df = pd.read_feather(file, columns=['title'])
        for line in tqdm(df['title'], desc='lines'):
            yield line

def build_vocabulary(files, save=False, filename='models/top_tfidf.pkd', **kwargs):
    tfv = TfidfVectorizer(**kwargs)
    tfv.fit(_iter_lines(files))
    if save:
        with open(filename, 'wb') as file:
            dill.dump(tfv, file)
    return tfv

In [4]:
folder = 'cleaned_cache'
files = [os.path.join(folder, 'data_{}.feather'.format(i)) for i in range(len(os.listdir(folder)))]

params = dict(min_df=10, ngram_range=(1,2), norm='l2', sublinear_tf=True, use_idf=True, max_features=5000)
tfv = build_vocabulary(files, **params)

In [5]:
filename='models/top_tfidf.pkd'

with open(filename, 'wb') as file:
    dill.dump(tfv, file)